In [1]:
from sklearn.metrics import confusion_matrix, classification_report

In [32]:
"""
INCOMPLETE, NOT READY TO BE RUN
"""
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten, Reshape
from modeling_utils import NumpyDataGenerator


# Parameters

def train_model(metadata_location,
                model_architecture = None,
                data_dim=(128,128),
                batch_size=64,
                n_classes=10,
                training_folds = [10,2,3,4,5,6,7,8,9],
                validation_folds = [1],
                shuffle=True):
    """
    """
    params = {'dim': data_dim,
              'batch_size': batch_size,
              'n_classes': n_classes,
              'shuffle': shuffle}

    # Datasets
    metadata = pd.read_csv(metadata_location)
    if shuffle:
        metadata = metadata.sample(frac=1)
    id_to_file_mapping = dict(zip(metadata['fsID'],metadata['location']))
    train_data = metadata[metadata['fold'].isin(training_folds)]
    test_data = metadata[metadata['fold'].isin(validation_folds)]
    # Generators
    training_generator = NumpyDataGenerator(list(train_data['fsID']), list(train_data['classID']),id_to_file_mapping, **params)
    validation_generator = NumpyDataGenerator(list(test_data['fsID']), list(test_data['classID']), id_to_file_mapping, **params)

    # Design model
    model = Sequential()
    model.add(Reshape((128*128,), input_shape=(128,128)))
    model.add(Dense(128, activation='relu',name='main_input'))
    model.add(Dense(64, activation='relu',name='second_layer'))
    model.add(Dense(32, activation='relu',name='middle_layer'))
    model.add(Dense(n_classes, activation='sigmoid',name='classifier'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    # Train model on dataset
    steps_per_epoch = np.ceil(len(metadata) / batch_size)
    validation_steps = np.ceil(len(list(test_data['classID']))/batch_size)
    model.fit_generator(generator=training_generator,
                        validation_data=validation_generator,
                        use_multiprocessing=True,
                        workers=6,
                        verbose=1,
                        steps_per_epoch=steps_per_epoch,
                        validation_steps=validation_steps,
                        epochs=1,
                        shuffle=True
                       )
    #Validate
    validation_generator_2 = NumpyDataGenerator(list(test_data['fsID']), list(test_data['classID']), id_to_file_mapping, **params)
    
    predictions = model.predict_generator(validation_generator_2,
                                      steps = validation_steps,
                                      verbose=True)
    y_pred = np.argmax(predictions, axis=1)
    print('Confusion Matrix')
    cm = confusion_matrix(list(test_data['classID']), y_pred[:len(list(test_data['classID']))])
    print(cm)
    print(np.unique(np.array(y_pred), return_counts=True))
    print(np.unique(np.array(test_data['classID']), return_counts=True))
    return train_data['classID'], test_data['classID']

In [33]:
metadata_file = "/media/romulo/6237-3231/urban_sound_challenge/spectrograms_128_array/location_mapping.csv"

In [34]:
train_model(metadata_file)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_6 (Reshape)          (None, 16384)             0         
_________________________________________________________________
main_input (Dense)           (None, 128)               2097280   
_________________________________________________________________
second_layer (Dense)         (None, 64)                8256      
_________________________________________________________________
middle_layer (Dense)         (None, 32)                2080      
_________________________________________________________________
classifier (Dense)           (None, 10)                330       
Total params: 2,107,946
Trainable params: 2,107,946
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1
101/101 [==============================] - 3s 27ms/step
Confusion Matrix
[[  0 800   0   0   0   0   0   0   0   0]
 

(6713     0
 34231    2
 7902     2
 53040    3
 8347     3
 8469     3
 13176    0
 60273    5
 51953    2
 39193    0
 43051    6
 52378    2
 20806    2
 39564    0
 9571     5
 20694    2
 49258    7
 48357    5
 23966    7
 10312    6
 35412    3
 9519     5
 34851    2
 16644    5
 36073    4
 23630    5
 20091    0
 51487    0
 44116    8
 22538    4
         ..
 34391    2
 37256    7
 61529    7
 61783    8
 25863    9
 11169    7
 43170    6
 17762    8
 53952    4
 59264    3
 43989    8
 29125    3
 62065    8
 45351    0
 37782    8
 27681    1
 13081    0
 40562    2
 51358    0
 22686    4
 52004    2
 9431     5
 30867    6
 56223    9
 50582    9
 37159    7
 29367    3
 39591    0
 35543    4
 53745    4
 Name: classID, Length: 56698, dtype: int64, 6028    9
 920     1
 5692    9
 4053    7
 5134    8
 1611    2
 1529    2
 5235    8
 4705    7
 717     0
 5218    8
 3992    6
 115     0
 6155    9
 2841    4
 3291    5
 2064    3
 1618    2
 456     0
 4826    7
 537